In [3]:
from KeyRecoveryScheme import h, p, Cipher
from leak import leak
import time
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad

In [4]:
# Credit to https://www.iacr.org/archive/eurocrypt2000/1807/18070053-new.pdf
# And to Keegan and Miro for helping me understand the attack, also teaching me some things about Sage

def lagrange_polynomial(i, x_lst, gf): # The polynomial L_i(x) according to the paper
    R.<X> = gf['X']
    result = 1
    for j in range(len(x_lst)):
        if (j == i):
            continue
        X_term = (X - x_lst[j]) / (x_lst[i] - x_lst[j])
        result *= X_term
    return result

def generate_A(pts, k, m, n, p):
    gf = GF(p)
    (x_lst, y_cand) = zip(*pts)
    for cand in y_cand:
        assert(len(cand) == m)
    assert(len(x_lst) == len(y_cand))
    assert(n == len(x_lst))
    assert(n > k + 1)
    # construct A as per p59 of Bleichenbacher & Nguyen
    A_rows = []
    for i in range(len(y_cand)):
        for j in range(len(y_cand[i])):
            l_i = lagrange_polynomial(i, x_lst, gf).coefficients(sparse=False)
            A_row = [(y_cand[i][j] * l_i[deg]) % p for deg in range(k+1, n)]
            A_rows.append(A_row) # append row to end
    A = matrix(GF(p), A_rows)
    return A

def generate_subspace(m, n, p): # m is number of candidates per x value, n is number of x values
    # credit to Keegan for the idea to express this rule as a kernel
    vectors = []
    for i in range(n - 1):
        left_i = i
        right_i = i + 1
        vector = [] # row in transposed matrix, col in the final matrix
        for i2 in range(n):
            for j2 in range(m):
                if (i2 == left_i):
                    vector.append(1)
                elif (i2 == right_i):
                    vector.append(-1)
                else:
                    vector.append(0)
        vectors.append(vector)
    return kernel(matrix(GF(p), vectors).transpose())

def get_pts(lock, possible_answers, m, n, p): # enumerates possibilities at each x coordinate, based on possible values of share at each coordinate
    s_i = [int(s_ij) for s_ij in lock.decode().split(",")]
    pts = []
    assert(len(s_i) == n)
    for x in range(0, n):
        assert(len(possible_answers[x]) == m)
        s_ij = s_i[x]
        cand = []
        for i in range(m):
            cip = Cipher((x + 1, possible_answers[x][i]), p)
            dec = cip.decrypt(s_ij)
            cand.append(dec)
        pts.append((x + 1, cand))
    return pts

def target_to_poly(target, pts, m, n, p):
    (x_lst, y_cand) = zip(*pts)
    assert(len(target) == m * n)
    index = 0
    gf = GF(p)
    R.<X> = GF(p)["X"]
    poly = 0
    for i in range(n):
        for j in range(m):
            poly += target[index] * y_cand[i][j] * lagrange_polynomial(i, x_lst, gf)
            index += 1
    return poly

def negative_vector(vector):
    return tuple(-1 * c for c in vector)

def valid_vector(vector, m, n, p):
    if (len(vector) != m * n): return False
    index = 0
    for i in range(n):
        j_sum = 0
        for j in range(m):
            if (vector[index] not in [0, 1]): return False
            j_sum += vector[index]
            index += 1
        if (j_sum != 1): return False
    return True

def get_lattice_basis(pts, k, m, n, p): # given noisy polynomial interpolation problem, generate lattice basis
    A = generate_A(pts, k, m, n, p)
    L = kernel(A)
    big_lambda = L.intersection(generate_subspace(m, n, p))
    big_lambda_matrix = matrix(ZZ, big_lambda.basis())
    p_identity_matrix = p * matrix.identity(ZZ, big_lambda_matrix.ncols())
    # credit to Keegan for teaching me this trick for lifting
    lifted_big_lambda_matrix = block_matrix([[big_lambda_matrix],[p_identity_matrix]])
    return lifted_big_lambda_matrix

def full_attack(lock, possible_answers, k, m, n, p):
    begin_time = time.time()
    pts = get_pts(lock, possible_answers, m, n, p)
    get_pts_time = time.time()
    print(f"Generated set of candidate y values for each x value ({get_pts_time - begin_time} seconds)")
    A = generate_A(pts, k, m, n, p)
    generate_A_time = time.time()
    print(f"Generated matrix A ({generate_A_time - get_pts_time} seconds)")
    L = kernel(A)
    kernel_A_time = time.time()
    print(f"Calculated L = kernel of A ({kernel_A_time - generate_A_time} seconds)")
    big_lambda = L.intersection(generate_subspace(m, n, p))
    intersection_time = time.time()
    print(f"Calculated improved sublattice of L ({intersection_time - kernel_A_time} seconds)")
    big_lambda_matrix = matrix(ZZ, big_lambda.basis())
    p_identity_matrix = p * matrix.identity(ZZ, big_lambda_matrix.ncols())
    # credit to Keegan for teaching me this trick for lifting
    lifted_big_lambda_matrix = block_matrix([[big_lambda_matrix],[p_identity_matrix]])
    lift_time = time.time()
    print(f"Lifted sublattice into ZZ ring ({lift_time - intersection_time} seconds)")
    reduced_basis = lifted_big_lambda_matrix.LLL()
    LLL_time = time.time()
    print(f"Reduced lattice basis ({LLL_time - lift_time} seconds)")
    for vec in reduced_basis.rows():
        is_valid = valid_vector(vec, m, n, p)
        is_valid_negative = valid_vector(negative_vector(vec), m, n, p)
        if (is_valid or is_valid_negative):
            if (is_valid):
                v = vec
            else:
                v = negative_vector(vec)
            print(f"Found target vector = {str(v)}")
            poly = target_to_poly(v, pts, m, n, p)
            key = h(poly.coefficients(sparse=False)[0])
            print(f"key = {str(key)}")
            final_time = time.time()
            print(f"Total time of full attack: {final_time - begin_time} seconds")
            return key
    print("Sorry, wasn't able to find target vector in reduced lattice basis :(")
    return reduced_basis.rows()

In [5]:
possible_answers = leak
possible_answers

[['thomas',
  'miku',
  'harris',
  'baltimore',
  'hipple',
  'ayer',
  'ashe',
  'arming',
  'acre',
  'beckinham'],
 ['russell',
  'tom',
  'jerry',
  'ashley',
  'johnny',
  'benjamin',
  'fred',
  'gerry',
  'dumbo',
  'sesame'],
 ['san diego',
  'san jose',
  'sacramento',
  'san francisco',
  'new york city',
  'los angeles',
  'san bernandino',
  'trenton',
  'detroit',
  'columbus'],
 ['birdemic',
  'fateful findings',
  'the last airbender',
  'troll 2',
  'batman and robin',
  'fantastic four',
  'disaster movie',
  'manos',
  'catwoman',
  'secrets of dumbledore'],
 ['decision to leave',
  'oldboy',
  'memories of murder',
  'blade runner 2049',
  'puss in boots 2',
  'into the spider-verse',
  'blade runner',
  'mother',
  'parasite',
  'portrait of a lady on fire'],
 ['barcarolle',
  'american boy',
  'gimme gimme gimme',
  'naatu naatu',
  'smells like teen spirit',
  'something in the way',
  'party in the usa',
  'thrift shop',
  'not afraid',
  'levels'],
 ['call me m

In [6]:
lock = b'190629033538511228468646206251559516519,238294487125633318381847448413858389148,27340971959422876078804955756782104475,201405021234643450057313886115967464005,73047844731805600891321835883882766999,4080439589675999097316970863193093516,171336484628641692271874903491662195159,215992970631530033943277468343898207908,75760716032030419139156362841569837182,101564264340921992580619866260582515217,81830384793133738498165413702028233091,39443721770710429698377379354299974512'
key = full_attack(lock=lock, possible_answers=possible_answers, k=10 - 1, m=10, n=12, p=p)

Generated set of candidate y values for each x value (0.0053141117095947266 seconds)
Generated matrix A (0.0561521053314209 seconds)
Calculated L = kernel of A (0.10942673683166504 seconds)
Calculated improved sublattice of L (0.7093510627746582 seconds)
Lifted sublattice into ZZ ring (0.009729623794555664 seconds)
Reduced lattice basis (4.046290636062622 seconds)
Sorry, wasn't able to find target vector in reduced lattice basis :(
